In [1]:
import pandas as pd
import networkx as nx
import networkx
import numpy as np
from matplotlib import pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
import pygraphviz as PG
import os,sys

In [2]:
root = '/Users/ajc/Core/PreProcessData/2020/Transport/'
nt_name = 'Network5aEdges'
path = os.path.join(root,'Synthetic', nt_name+ '.csv')
Df = pd.read_csv(path,sep=',')
#print (Df)
#tree_data = pd.DataFrame(columns=Df.columns)

tree_data = dict()
for c in Df.columns: 
    d1 =  [x  for x in Df[c] if not np.isnan(x)]
    #print (c)
    tree_data[c] = np.array(d1)
    
#print (tree_data)

In [3]:
def get_pairs():
    d = [(int(x),int(y)) for x,y in zip(tree_data['downstream'],tree_data['upstream'])]
    return d

In [4]:
pairs = get_pairs()
#print (pairs)
#pairs = [(1, 2),(2, 3),(3, 4), (3, 5),(5, 9),(5, 7), (2, 8), (8,10), (8,11)]

graph_syn = nx.from_edgelist(pairs)
#print (graph_syn.nodes)
labels = graph_syn.nodes
Labels ={}
for l in labels:
    Labels[l] = l

In [5]:
%matplotlib tk
GraphPG = PG.AGraph()

for i,p in enumerate(pairs):
    #print (tree_data['length'][i])
    c = str(tree_data['length'][i]/1000)
    if i == 0:
        GraphPG.add_edge(p[0], p[1], len=c)
    else:
        GraphPG.add_edge(p[0], p[1], len=c)#tree_data['length'][i])

fig, ax = plt.subplots()
filename = os.path.join(root,'Synthetic/') + nt_name + '.png'
GraphPG.draw(filename, format='png', prog='neato')

In [6]:
%matplotlib tk
Graph = nx.from_edgelist(pairs)

pos = nx.spring_layout(Graph, iterations=10,weight='weight')
nx.draw(Graph, pos)#=nx.spring_layout(Graph))

nx.draw_networkx_labels(Graph,pos,Labels,font_size=10)

plt.axis('off')
plt.show() # display

/Users/ajc/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


In [7]:
import math

f_vector = lambda p,q: [q[0] - p[0], q[1] - p[1],  q[2] - p[2]]
f_distance = lambda p,q: np.sqrt ((p[0] - q[0])**2 + (p[1] - q[1])**2 + (p[2] - q[2])**2)

f_normal = lambda v,d: [v[0]/d, v[1]/d, v[2]/d]

f_slope = lambda p,q: (q[1] - p[1])/(q[0] - p[0])

def get_normal(coords):
    pos_v = f_vector(coords[0], coords[1])
    distance = math.sqrt(pos_v[0]**2 + pos_v[1]**2 + pos_v[2]**2)
    normal = f_normal(pos_v, distance)
    normal = [round(n,6) for n in normal]
    return normal
def get_segment_coords(coords,cells_length):
    #print (coords,cells_length)
    P,Q = coords[0],coords[1]
    m = f_slope(P,Q)
    slope = False
    if np.isinf(m):
        m = 0
        slope = True
    seg_coords = []
    seg_coords.append(P)
    print ("get_segment_coord:: ", P,Q,m)
    for i,x in enumerate(cells_length):
        
        y = P[1] + m*(x)
        #print ('X,Y: ',x+P[0],y)   
        if slope == True:
            y = P[1] + 1*(x)
            seg_coords.append([P[0],y])
            print ('X,Y: ',x,y, P[0], P[1]) 
        else:
            seg_coords.append([P[0]+x,y])
         
        
    return seg_coords

In [8]:
def cell_lengths_and_centroids_simple(length_,coords_,spacing='Uniform',x_spacing=25.):
    length = length_    
    
    if (spacing == "Uniform"):
        d = int(length/x_spacing)
        dx = length/d # ensures all dx sum to the total length
        cells_spacing = np.round(np.arange(0,length+dx,dx),4)
        #print ('C ',cells_spacing)
        cells_length = list(cells_spacing[1:]) # first cell entity is ignored, zero length
    
    ncells = len(cells_length) if len(cells_length) > 0 else 1
    
    sum1 = 0

    
    seg_coords = get_segment_coords(coords_,cells_length)
    
    centroids = []
    #print ('Coords: ',seg_coords)
    for i in range(1,len(seg_coords),1):
        cent = [0.5* (seg_coords[i-1][0] + seg_coords[i][0]), 0.5* (seg_coords[i-1][1] + seg_coords[i][1]), 0]
        #print ('Centroids: ', cent)#,seg_coords[i-1],seg_coords[i])
        centroids.append(cent)
    
    centroids = list(np.concatenate(centroids))
    cells_spacing = list(cells_spacing)
    cells_length1 = [cells_spacing[0],]*len(cells_spacing)
    assert (len(centroids) == 3*len(cells_length))
    #print (cells_length)
    return centroids, cells_length, len(cells_length)

In [9]:
#c1 = cell_lengths_and_centroids_simple(length_=1000,coords_=coords,spacing='Uniform',x_spacing=25)
#c1

In [10]:
DataFlow = dict()
discharge = []
velocity = []

Normal = []
segment_depth = []
segment_length = []
segment_width = []
segment_crossA = []

km_to_m = 1000.
cfs_to_ms = 0.028316847 # QAMA is in cfs, convert cfs to m/s
fs_to_ms = 0.3048

# get discharge, velocity, depth, and orientation
print (len(Graph))
def River_Characteristics():
    d4 = []
    d5 = []
    
    for k in range(len(tree_data['Q'])):
      
        id = k
        value_q = tree_data['Q'][id]
        discharge.append([('ID',id), ('Discharge [m^3/s]',value_q)])
                
        length = tree_data['length'][id]
        cross_area = tree_data['width'][id] * tree_data['depth'][id]
        
        width = 1.0
        depth = tree_data['depth'][id]
                
        segment_length.append([('ID',id), ('segment length [m]', length)])
        segment_depth.append([('ID',id), ('segment depth [m]', depth)])
        segment_width.append([('ID',id), ('segment width [m]', width)])
        segment_crossA.append([('ID',id), ('segment cross area [m]', cross_area)])
                
        # orientation
        #coords = [tree_data['coordx n0']
        down,up = int(tree_data['downstream'][id]),int(tree_data['upstream'][id])
        #print (down,up)
        p = [tree_data['x'][down-1], tree_data['y'][down-1],0]
        q = [tree_data['x'][up-1], tree_data['y'][up-1],0]
        #print ('PQ: ',p,q, down,up)
        #print ('----------------------------------------')
        n = get_normal([p,q])
        
        
        Normal.append([n[0],n[1],n[2]])
        
        #n = [1,1,0]
        d4.append([('ID',id), ('Discharge [m^3/s]',value_q), ('segment length [m]', length), 
                   ('segment cross area [m]', cross_area), ('orientation', [n[0],n[1],n[2]] )])
    
        
       
        centroids, cell_lengths, ncells = cell_lengths_and_centroids_simple(length_=length,coords_=[p,q],spacing='Uniform',x_spacing=100)
        d5.append([('ID',id), ('centroids', centroids),
                   ('cell lengths [m]', cell_lengths), ('number of cells', ncells)])
        #print (len(centroids), ncells)
        
    return d4, d5



492


In [11]:
Data_ATS, DataMesh_ATS  = River_Characteristics()

/Users/ajc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  


get_segment_coord::  [0.0, 0.0, 0] [1867.0, 0.0, 0] 0.0
get_segment_coord::  [1867.0, 0.0, 0] [3733.0, 0.0, 0] 0.0
get_segment_coord::  [3733.0, 0.0, 0] [5600.0, 0.0, 0] 0.0
get_segment_coord::  [5600.0, 0.0, 0] [7467.0, 0.0, 0] 0.0
get_segment_coord::  [7467.0, 0.0, 0] [9333.0, 0.0, 0] 0.0
get_segment_coord::  [9333.0, 0.0, 0] [11200.0, 0.0, 0] 0.0
get_segment_coord::  [11200.0, 0.0, 0] [13100.0, 0.0, 0] 0.0
get_segment_coord::  [13100.0, 0.0, 0] [14900.0, 0.0, 0] 0.0
get_segment_coord::  [14900.0, 0.0, 0] [16800.0, 0.0, 0] 0.0
get_segment_coord::  [16800.0, 0.0, 0] [18700.0, 0.0, 0] 0.0
get_segment_coord::  [18700.0, 0.0, 0] [20500.0, 0.0, 0] 0.0
get_segment_coord::  [20500.0, 0.0, 0] [22400.0, 0.0, 0] 0.0
get_segment_coord::  [22400.0, 0.0, 0] [24300.0, 0.0, 0] 0.0
get_segment_coord::  [24300.0, 0.0, 0] [26100.0, 0.0, 0] 0.0
get_segment_coord::  [26100.0, 0.0, 0] [28000.0, 0.0, 0] 0.0
get_segment_coord::  [28000.0, 0.0, 0] [29900.0, 0.0, 0] 0.0
get_segment_coord::  [29900.0, 0.0, 0]

X,Y:  2270.0741 -11629.9259 3733.0 -13900.0
X,Y:  2373.2593 -11526.7407 3733.0 -13900.0
X,Y:  2476.4444 -11423.5556 3733.0 -13900.0
X,Y:  2579.6296 -11320.3704 3733.0 -13900.0
X,Y:  2682.8148 -11217.1852 3733.0 -13900.0
X,Y:  2786.0 -11114.0 3733.0 -13900.0
get_segment_coord::  [3733.0, -16700.0, 0] [3733.0, -19500.0, 0] 0
X,Y:  103.1852 -16596.8148 3733.0 -16700.0
X,Y:  206.3704 -16493.6296 3733.0 -16700.0
X,Y:  309.5556 -16390.4444 3733.0 -16700.0
X,Y:  412.7407 -16287.2593 3733.0 -16700.0
X,Y:  515.9259 -16184.0741 3733.0 -16700.0
X,Y:  619.1111 -16080.8889 3733.0 -16700.0
X,Y:  722.2963 -15977.7037 3733.0 -16700.0
X,Y:  825.4815 -15874.5185 3733.0 -16700.0
X,Y:  928.6667 -15771.3333 3733.0 -16700.0
X,Y:  1031.8519 -15668.1481 3733.0 -16700.0
X,Y:  1135.037 -15564.963 3733.0 -16700.0
X,Y:  1238.2222 -15461.7778 3733.0 -16700.0
X,Y:  1341.4074 -15358.5926 3733.0 -16700.0
X,Y:  1444.5926 -15255.4074 3733.0 -16700.0
X,Y:  1547.7778 -15152.2222 3733.0 -16700.0
X,Y:  1650.963 -15049.037 

get_segment_coord::  [55100.0, 10300.0, 0] [56300.0, 11500.0, 0] 1.0
get_segment_coord::  [56300.0, 11500.0, 0] [57400.0, 12600.0, 0] 1.0
get_segment_coord::  [57400.0, 12600.0, 0] [58600.0, 13800.0, 0] 1.0
get_segment_coord::  [16800.0, 7800.0, 0] [25300.0, 7800.0, 0] 0.0
get_segment_coord::  [16800.0, 15600.0, 0] [17900.0, 16700.0, 0] 1.0
get_segment_coord::  [16800.0, 3900.0, 0] [15700.0, 5031.0, 0] -1.028181818181818
get_segment_coord::  [16800.0, 11700.0, 0] [15700.0, 12800.0, 0] -1.0
get_segment_coord::  [16800.0, 19500.0, 0] [13800.0, 22500.0, 0] -1.0
get_segment_coord::  [13800.0, 22500.0, 0] [10800.0, 25500.0, 0] -1.0
get_segment_coord::  [16800.0, 19500.0, 0] [18000.0, 20700.0, 0] 1.0
get_segment_coord::  [18000.0, 20700.0, 0] [19200.0, 21900.0, 0] 1.0
get_segment_coord::  [19200.0, 21900.0, 0] [20400.0, 23100.0, 0] 1.0
get_segment_coord::  [20400.0, 23100.0, 0] [21600.0, 24300.0, 0] 1.0
get_segment_coord::  [21600.0, 24300.0, 0] [22800.0, 25500.0, 0] 1.0
get_segment_coord:: 

get_segment_coord::  [10800.0, 25500.0, 0] [8940.0, 25500.0, 0] -0.0
get_segment_coord::  [8940.0, 25500.0, 0] [7090.0, 25500.0, 0] -0.0
get_segment_coord::  [19200.0, 21900.0, 0] [18800.0, 22300.0, 0] -1.0
get_segment_coord::  [18800.0, 22300.0, 0] [18500.0, 22700.0, 0] -1.3333333333333333
get_segment_coord::  [18500.0, 22700.0, 0] [18100.0, 23000.0, 0] -0.75
get_segment_coord::  [18100.0, 23000.0, 0] [17700.0, 23400.0, 0] -1.0
get_segment_coord::  [17700.0, 23400.0, 0] [17300.0, 23800.0, 0] -1.0
get_segment_coord::  [17300.0, 23800.0, 0] [17000.0, 24200.0, 0] -1.3333333333333333
get_segment_coord::  [17000.0, 24200.0, 0] [16600.0, 24500.0, 0] -0.75
get_segment_coord::  [21600.0, 24300.0, 0] [21600.0, 25900.0, 0] 0
X,Y:  100.0 24400.0 21600.0 24300.0
X,Y:  200.0 24500.0 21600.0 24300.0
X,Y:  300.0 24600.0 21600.0 24300.0
X,Y:  400.0 24700.0 21600.0 24300.0
X,Y:  500.0 24800.0 21600.0 24300.0
X,Y:  600.0 24900.0 21600.0 24300.0
X,Y:  700.0 25000.0 21600.0 24300.0
X,Y:  800.0 25100.0 21

X,Y:  900.0 -3015.0 55900.0 -3915.0
X,Y:  1000.0 -2915.0 55900.0 -3915.0
X,Y:  1100.0 -2815.0 55900.0 -3915.0
X,Y:  1200.0 -2715.0 55900.0 -3915.0
X,Y:  1300.0 -2615.0 55900.0 -3915.0
X,Y:  1400.0 -2515.0 55900.0 -3915.0
X,Y:  1500.0 -2415.0 55900.0 -3915.0
X,Y:  1600.0 -2315.0 55900.0 -3915.0
get_segment_coord::  [57100.0, -2713.0, 0] [58700.0, -2713.0, 0] 0.0
get_segment_coord::  [58300.0, -1511.0, 0] [58300.0, 339.4, 0] 0
X,Y:  102.7778 -1408.2222 58300.0 -1511.0
X,Y:  205.5556 -1305.4444 58300.0 -1511.0
X,Y:  308.3333 -1202.6667 58300.0 -1511.0
X,Y:  411.1111 -1099.8889 58300.0 -1511.0
X,Y:  513.8889 -997.1111 58300.0 -1511.0
X,Y:  616.6667 -894.3333 58300.0 -1511.0
X,Y:  719.4444 -791.5556 58300.0 -1511.0
X,Y:  822.2222 -688.7778 58300.0 -1511.0
X,Y:  925.0 -586.0 58300.0 -1511.0
X,Y:  1027.7778 -483.22219999999993 58300.0 -1511.0
X,Y:  1130.5556 -380.4444000000001 58300.0 -1511.0
X,Y:  1233.3333 -277.6667 58300.0 -1511.0
X,Y:  1336.1111 -174.88889999999992 58300.0 -1511.0
X,Y:  1

X,Y:  100.0 10500.0 27900.0 10400.0
X,Y:  200.0 10600.0 27900.0 10400.0
X,Y:  300.0 10700.0 27900.0 10400.0
X,Y:  400.0 10800.0 27900.0 10400.0
X,Y:  500.0 10900.0 27900.0 10400.0
X,Y:  600.0 11000.0 27900.0 10400.0
X,Y:  700.0 11100.0 27900.0 10400.0
X,Y:  800.0 11200.0 27900.0 10400.0
X,Y:  900.0 11300.0 27900.0 10400.0
X,Y:  1000.0 11400.0 27900.0 10400.0
X,Y:  1100.0 11500.0 27900.0 10400.0
X,Y:  1200.0 11600.0 27900.0 10400.0
X,Y:  1300.0 11700.0 27900.0 10400.0
X,Y:  1400.0 11800.0 27900.0 10400.0
X,Y:  1500.0 11900.0 27900.0 10400.0
X,Y:  1600.0 12000.0 27900.0 10400.0
get_segment_coord::  [26600.0, 6492.0, 0] [26600.0, 4892.0, 0] 0
X,Y:  100.0 6592.0 26600.0 6492.0
X,Y:  200.0 6692.0 26600.0 6492.0
X,Y:  300.0 6792.0 26600.0 6492.0
X,Y:  400.0 6892.0 26600.0 6492.0
X,Y:  500.0 6992.0 26600.0 6492.0
X,Y:  600.0 7092.0 26600.0 6492.0
X,Y:  700.0 7192.0 26600.0 6492.0
X,Y:  800.0 7292.0 26600.0 6492.0
X,Y:  900.0 7392.0 26600.0 6492.0
X,Y:  1000.0 7492.0 26600.0 6492.0
X,Y:  1100.

X,Y:  1300.0 -11800.0 67700.0 -13100.0
X,Y:  1400.0 -11700.0 67700.0 -13100.0
X,Y:  1500.0 -11600.0 67700.0 -13100.0
X,Y:  1600.0 -11500.0 67700.0 -13100.0
get_segment_coord::  [68400.0, -12500.0, 0] [70000.0, -12500.0, 0] 0.0
get_segment_coord::  [69100.0, -11800.0, 0] [69100.0, -10200.0, 0] 0
X,Y:  100.0 -11700.0 69100.0 -11800.0
X,Y:  200.0 -11600.0 69100.0 -11800.0
X,Y:  300.0 -11500.0 69100.0 -11800.0
X,Y:  400.0 -11400.0 69100.0 -11800.0
X,Y:  500.0 -11300.0 69100.0 -11800.0
X,Y:  600.0 -11200.0 69100.0 -11800.0
X,Y:  700.0 -11100.0 69100.0 -11800.0
X,Y:  800.0 -11000.0 69100.0 -11800.0
X,Y:  900.0 -10900.0 69100.0 -11800.0
X,Y:  1000.0 -10800.0 69100.0 -11800.0
X,Y:  1100.0 -10700.0 69100.0 -11800.0
X,Y:  1200.0 -10600.0 69100.0 -11800.0
X,Y:  1300.0 -10500.0 69100.0 -11800.0
X,Y:  1400.0 -10400.0 69100.0 -11800.0
X,Y:  1500.0 -10300.0 69100.0 -11800.0
X,Y:  1600.0 -10200.0 69100.0 -11800.0
get_segment_coord::  [69700.0, -11200.0, 0] [71300.0, -11200.0, 0] 0.0
get_segment_coord:

In [12]:
len(Data_ATS)

491

In [13]:
def treeDiagnostic(graph):
    for node in graph:
        edges,_ = node_get_upstream_edges(graph,[node])
        for edge in edges:
            print (edge)
            assert edge[0] < edge[1]

def numNodes(graph):
    return len(graph.nodes)


def isNeighborNode(graph,node):
    ngh_nodes = list(graph.neighbors(node))
    return  True if len(ngh_nodes) > 0 else False

def node_get_edges(graph, node):
    
    edges = list(nx.edge_boundary(graph, node))
    return edges

def node_get_upstream_edges(graph, node):
    edges = list(nx.edge_boundary(graph, node))
    parent = -100
    for i,edge in enumerate(edges):
        if edge[0] > edge[1]:
            parent = edges[i]
            edges.pop(i)
    return edges, parent

def isNeighborBoundaryNode(graph, edge):
    ngh = list(graph.neighbors(edge[1]))
    return False if len(ngh) > 1 else True

def isBoundaryNode(graph, node):
    assert (node != 1)
    ngh = list(graph.neighbors(node))
    return False if len(ngh) > 1 else True

def get_node_index(graph, node):
    x = np.where(np.array(graph.nodes) == node)[0][0] + 1
    return x

def index_based_graph(graph):
    Nodes_new_ids = np.linspace(1,len(graph),len(graph), dtype='i')
    Nodes = graph.nodes

def applied_source(Q, Contri_A):
    g = 1020
    h = 0.345
    rho_molar = 55555.55
    rho_water = 1000
    mass_molar = rho_water / rho_molar  #[kg/moles]
    Qreach = Q[-1] # discharge at the first order stream outlet
    Source = np.zeros(len(Q)-1)
    for i in range(len(Q)-1):
        del_q = Q[i+1] - Q[i]
        C = g * (Qreach / (Contri_A * 8.64E+8) )**h
        Source[i] = (del_q * C) / mass_molar # conver source [kg/sec] to [moles/sec]
        #print (C, del_q, Q[i+1], Q[i])
        
    return Source
    
    

In [14]:
from lxml import etree
import lxml.builder
import lxml.etree as ET

#num_nodes = numNodes(Graph)
fine_roots = []

Discharge =  []
CrossSectionArea = []
BoundaryCells = []
BoundaryReachCells = []
Total_cells = 0
def WriteXMLNHD_Centroid():
    global Total_cells
    index = 0
    root = ET.Element("ParameterList", name="Main")
    ET.SubElement(root,"Parameter",name="infer cell centroids",type="bool", value="true")
    segments = ET.SubElement(root,"ParameterList", name="segments")
    #---- for regions
    root_region = ET.Element("ParameterList", name="Main")
    #--------
    nam = ''
    node_id_root = 0
    total_cells = 0
    bc_cell_id = 0
    tt = 0
    for i,node in enumerate(Graph):
        #print ('Top ',i, len(Graph)-1)#, Data_ATS[i][3])
        if (i == len(Graph)-1):
            print ('Break')
            break
        
        length =  Data_ATS[i][2][1]
        width = 1
        orientation = Data_ATS[i][4][1]
        cross_area = Data_ATS[i][3][1]
        
        
        centroids = DataMesh_ATS[i][1][1]
        cell_lengths = DataMesh_ATS[i][2][1]
        ncells = DataMesh_ATS[i][3][1]
        
        total_cells = total_cells + ncells
        
        if i ==0:
            index = 0
            node_id_root = node
                       
            edges = node_get_edges(Graph,[node])[0]
            
            nam = 'stream_' + str(edges[0]) + '_' + str(edges[1])
           
            streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
            ET.SubElement(streams, "Parameter", name="segment length [m]", type="double", value="%s"%length)
            
            ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="%s"%ncells)
            
            n1,n2 = orientation[0],orientation[1]
            
            ET.SubElement(streams,"Parameter",name="orientation", type="Array(double)", value="{%s,%s,0}"%(n1,n2))
            ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="%s"%cross_area)
            ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="boundary")
            ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="junction")
            ET.SubElement(streams,"Parameter",name="cell centroids", type="Array(double)", value="{%s}"% ", ".join(map(str,centroids)))
            #kk = ["{%s}"%c for c in cell_lengths]
            #s = ", ".join(map(str,  cell_lengths))
            #print (s)
            ET.SubElement(streams,"Parameter",name="cell lengths [m]", type="Array(double)", value="{%s}"% ", ".join(map(str,  cell_lengths)))
            
            Discharge.append(Data_ATS[index][1][1])
            #print (Discharge)
            #break
            CrossSectionArea.append(Data_ATS[index][3][1])
            bc_cell_id = bc_cell_id + ncells
            tt  = tt + ncells
            r_ids = [-1 for i1 in range(ncells)] 
            BoundaryReachCells.append(r_ids)
            #print (ncells)
        else:
            edges, parent = node_get_upstream_edges(Graph,[node])
           
            edge_len = len(edges)
            
            if edge_len == 0: #boundary edge
                #print ('Edge', i, edge_len)
                continue
                
            for j, child_node in enumerate(edges):
                
                if child_node[1] != node_id_root:
                    index = child_node[1] - 2 
                    index_parent = child_node[0] - 2 
                    nam = 'stream_' + str(child_node[0]) + '_' + str(child_node[1])
                    streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
                   
                    centroids = DataMesh_ATS[index][1][1]
                    cell_lengths = DataMesh_ATS[index][2][1]
                    ncells = DataMesh_ATS[index][3][1]
                    cross_area = Data_ATS[index][3][1]
                    length = Data_ATS[index][2][1]
                    
                    ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="branch")
                    ET.SubElement(streams,"Parameter",name="first tip branch segment", type="string", value="stream_%s_%s"%(parent[1],parent[0]))
                    ET.SubElement(streams,"Parameter",name="first tip branch segment tip", type="string", value="last")
                    
                    if isBoundaryNode(Graph,child_node[1]):
                        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="boundary")
                    else:
                        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="junction")
                        
                    n1,n2 = orientation[0],orientation[1]
                    
                    ET.SubElement(streams,"Parameter",name="orientation", type="Array(double)", value="{%s,%s,0}"%(n1,n2))
                    ET.SubElement(streams, "Parameter", name="segment length [m]", type="double", value="%s"%length)
                    ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="%s"%ncells) 
                    ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="%s"%cross_area)
                    ET.SubElement(streams,"Parameter",name="cell centroids", type="Array(double)", value="{%s}"%", ".join(map(str,centroids)))
                    ET.SubElement(streams,"Parameter",name="cell lengths [m]", type="Array(double)", value="{%s}"% ", ".join(map(str,cell_lengths)))
                    
                    fine_roots.append(nam)
                    Discharge.append(Data_ATS[index][1][1])
                    CrossSectionArea.append(Data_ATS[index][4][1])
                    
                    bc_cell_id = bc_cell_id + ncells
                    # get boundary nodes ids
                    tt  = tt + ncells
                    #print (ncells)
                    if isBoundaryNode(Graph,child_node[1]):
                        
                        BoundaryCells.append(bc_cell_id-1)
                        r_ids = [bc_cell_id -1 - i1 for i1 in range(ncells)] 
                        BoundaryReachCells.append(r_ids)
                        #print (nam, Discharge[-1],ncells,BoundaryReachCells[-1])
                        reach_region = ET.SubElement(root_region,"ParameterList", name="%s"%nam)
                        ET.SubElement(reach_region,"Parameter",name="entity", type="string", value="face")
                        ET.SubElement(reach_region,"Parameter",name="entity gids", type="Array(int)", value="%s"%r_ids)
                    else:
                        r_ids = [-1 for i1 in range(ncells)] 
                        BoundaryReachCells.append(r_ids)
                    i = i + j
                    #print ('Enter',i)
                #
            #
        #
        Total_cells = tt
        #print ('Out',i, node,len(Graph))#, Data_ATS[i][3])
        if (i+2 >= len(Graph)):
            break
    sets = ET.SubElement(root,"ParameterList",name="sets",type="bool", value="true")
    ET.SubElement(sets,"Parameter",name="fine_root",type="Array(string)", value="{%s}"%", ".join(map(str,fine_roots)))
    Data= etree.tostring(root,encoding="unicode",pretty_print=True)
    Data_regions = etree.tostring(root_region,encoding="unicode",pretty_print=True)
    
    outfile= open('mesh_synthetic_5order.xml','w')
    outfile.write(Data)
    
    #write xml file for 1st order stream
    #outfile= open('syn-reaches.xml','w')
    #outfile.write(Data_regions)






In [15]:

WriteXMLNHD_Centroid()

Break
